In [1]:
import pandas as pd


In [35]:
training_df = pd.read_csv("./project/training_data.csv")

# df_train.describe()
print(training_df)


                                                   title  \
0                                       The Three Amigos   
1      Home Essentials Blue Floral Glass Vintage Styl...   
2      Cooper Wiring Quiet Toggle Switch Single Pole ...   
3                        Baseboarders&reg; Wall Brackets   
4      The Great Wave Off Kanagawa Custom Rectangle M...   
...                                                  ...   
20183  D0309-WML Leviton Decora Screwless Snap-On Wal...   
20184  South Shore Axess Small 2-Door Storage Cabinet...   
20185       Patriotic Flip Flop Ornament Assortment of 2   
20186  Cybrtrayd Life of the Party A116 Horse Chocola...   
20187  4 Light Track Lighting Wall &amp; Ceiling Moun...   

                       category  
0                          None  
1                Home & Kitchen  
2      Tools & Home Improvement  
3      Tools & Home Improvement  
4               Office Products  
...                         ...  
20183                      None  
20184  

In [34]:
training_df['category'].value_counts()

None                        10123
Home & Kitchen               4960
Tools & Home Improvement     2080
Office Products              1144
Grocery & Gourmet Food       1102
Industrial & Scientific       588
Electronics                   191
Name: category, dtype: int64

In [36]:
final_df = training_df[training_df["category"] != "None"]
final_df

,title,category
1,Home Essentials Blue Floral Glass Vintage Styl...,Home & Kitchen
2,Cooper Wiring Quiet Toggle Switch Single Pole ...,Tools & Home Improvement
3,Baseboarders&reg; Wall Brackets,Tools & Home Improvement
4,The Great Wave Off Kanagawa Custom Rectangle M...,Office Products
5,Nemcor Pittsburgh Penguins NHL #1 Ultimate Fan...,Home & Kitchen
...,...,...
20175,Curtain Chic Supple Microfiber Cumberbund Vala...,Home & Kitchen
20176,Jack &amp; Jason's Pancake &amp; Waffle Mix - ...,Grocery & Gourmet Food
20178,4 Swivel Casters 4&quot; Rubber Double Brake B...,Industrial & Scientific
20185,Patriotic Flip Flop Ornament Assortment of 2,Home & Kitchen


In [17]:
import re
def normalize_text(s):
    s = s.lower()
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s = re.sub('\s\W', ' ', s)
    s = re.sub('\W\s', ' ', s)

    # make sure we didn't introduce any double spaces
    s = re.sub('\s+', ' ', s)
    return s

In [18]:
ip = list(map(normalize_text, final_df["title"]))
ip

['home essentials blue floral glass vintage style water goblet set of 4',
 'cooper wiring quiet toggle switch single pole lighted 15 a 120 v ivory ul boxed',
 'baseboarders&reg wall brackets',
 'the great wave off kanagawa custom rectangle mouse pad oblong gaming mousepad in 220mm*180mm*3mm 9&quot;*7&quot; 912051',
 'nemcor pittsburgh penguins nhl 1 ultimate fan black/gold finger style throw cushion pillow',
 '20 qty halco 50w mr16 fl 24v gu5.3 prism exn mr16exn/24v 50w 24v halogen flood lamp bulb',
 'redbirdlinen 1pc brown sheer valance scarf 216&quot long beautiful elegant voile topper',
 'hospital bath towel 24x48in. 8 lb/dz pack of 12',
 'symphony in red and khaki by laurie maitland 29x29',
 'big train blended ice pumpkin spice 3.5 lbs bag)',
 'surpahs round 11 lb/5 kg digital kitchen and food scale w small removable bowl',
 'anti-slip handle for 30 oz yeti rambler tumbler fits 30 oz rtic,kodiak and more free extral long stainless steel drinking straw cleaning brush handle only-tum

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
ip = vectorizer.fit_transform(ip)
ip

<10065x21376 sparse matrix of type '<class 'numpy.int64'>'
	with 110774 stored elements in Compressed Sparse Row format>

In [20]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
op = encoder.fit_transform(final_df["category"])
op

array([2, 5, 5, ..., 3, 2, 5])

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(ip, op, test_size=0.2)
y_test

array([3, 5, 1, ..., 3, 5, 1])

In [25]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train, y_train)
print("Accuracy: ", nb.score(x_test, y_test))

Accuracy:  0.8132141082960755


In [26]:
def predict_category(index):
    return encoder.classes_[index]

In [27]:
def get_predicated(s):
    processed_ip = normalize_text(s)
    vectorized = vectorizer.transform([processed_ip])
    category = nb.predict(vectorized)
    return predict_category(category[0])

In [37]:
test_df = pd.read_csv('./project/testing_data.csv')
print(test_df)
final_data = pd.DataFrame(columns=["title", "predicted_category"])
final_data

                                                   title  category
0      Pom Pom Hair Band Rabbit Light Grey Decorative...       NaN
1                       Mariposa Golf Ball Napkin Weight       NaN
2      Mediterranean Snack Food Roasted Garlic Hummuz...       NaN
3      John Deere 0071750GX22269 Genuine Original Equ...       NaN
4      Protech Wood Cleanr Gl,PERFORMANCE COATINGS IN...       NaN
...                                                  ...       ...
10089  Flexible Coupling - 3.0&quot; x 3.0&quot; - White       NaN
10090  Mr. Christmas 4 1/2-Inch Mini Porcelain Music ...       NaN
10091  Jewish New Year - Shanah Tovah - Rosh Hashana ...       NaN
10092  Hotel Collection Gridwork Queen Bedskirt Graphite       NaN
10093  2-LBS SUPER FRESH ROASTED &ndash; YIRGACHEFFE ...       NaN

[10094 rows x 2 columns]


,title,predicted_category


In [30]:
def add_predicted_data(title):
    final_data.loc[len(final_data.index)] = [title, get_predicated(title)]

In [31]:
for i in test_df["title"]:
    add_predicted_data(i)
    
final_data.to_csv('predicted_classes.csv')